In [289]:
driver = webdriver.Chrome()

driver.get(url)
driver.implicitly_wait(3)

html_page = driver.page_source
driver.quit()

soup = BeautifulSoup(html_page, 'lxml')
reviews = soup.findAll('div', attrs={"data-hook": "review"})

rev = []
for review in reviews:
    name = review.find('span', class_='a-profile-name').text.strip()    

    # Amazon's format is "x.0 stars out of 5" where x = # of stars
    rating = review.find('span', class_='a-icon-alt').text.strip()[0]

    review = review.find('span', attrs={'data-hook': 'review-body'}).text.strip()

    rev.append({'customer_name': name, 'rating': rating,'review': review})

# for customer name, find nested span class="a-profile-name"
# for customer rating, find nested span class="a-icon-alt"
# for customer review, find nested span data-hook="review"

In [293]:
for review in rev:
    print(review)

{'customer_name': 'Hayley', 'rating': '1', 'review': 'To soft'}
{'customer_name': 'Rohit', 'rating': '1', 'review': 'I had bought a refurbished iPhone 14 that was unlocked and was having troubles setting up the eSIM (because iphone14 and up are eSIM only) through various trials and errors we had looked up the IEMI number up on T-Mobile data base and it had been reported as stolen through xfinity, now I have a phone that I can’t uses because of this, unbelievable.'}
{'customer_name': 'Krystal', 'rating': '1', 'review': "This was the worst decision. I got the phone and the screen did not work. The screen is not apple and can tell it doesn't correctly fit. Even if I replaced it, T-Mobile said I would have the same problem. Don't buy from 3rd party. This was a scam."}
{'customer_name': 'Zhangbaoquan', 'rating': '1', 'review': 'The screen often goes black and the phone freezes. This phone must have been severely dropped or soaked in water. It is unethical to sell such a phone.'}
{'customer_

In [256]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [274]:
def get_rated_reviews(url: str, num_reviews: int):
    driver = webdriver.Chrome()

    # make browser headless so it works in the background
    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver.get(url)
    driver.implicitly_wait(3)

    html_page = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html_page, 'lxml')
    reviews = soup.findAll('span', attrs={'data-hook': True})

    print(reviews)

    reviews = [review for review in reviews if review['data-hook'] == 'review-body']

    # print('\n\n\n\n\n')
    # print(reviews)

    # extract text from span tag and clean up newlines in string
    reviews = [review.text.strip() for review in reviews]    

    return reviews

In [273]:
get_rated_reviews('https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=2
', 3)

[<span class="a-size-medium a-color-base" data-hook="rating-out-of-text">4.2 out of 5</span>, <span class="a-size-base review-title a-text-bold" data-hook="review-title">Never Disappointed</span>, <span class="a-size-base review-title a-text-bold" data-hook="review-title">To soft</span>, <span class="a-text-bold" data-hook="cr-filter-info-text" id="reviews-filter-info-segment">1 star</span>, <span class="a-size-base review-title a-color-base review-title-content a-text-bold" data-hook="review-title">
<span>The phone never arrived. Only the box.</span>
</span>, <span class="a-size-base a-color-secondary review-date" data-hook="review-date">Reviewed in the United States on March 23, 2024</span>, <span class="a-size-mini a-color-state a-text-bold" data-hook="avp-badge">Verified Purchase</span>, <span class="a-size-base review-text review-text-content" data-hook="review-body">
<span>Purchased a renewed phone and the box came with the charger and cable and no phone.</span>
</span>, <span cl

['Purchased a renewed phone and the box came with the charger and cable and no phone.',
 'The screen comes up with the following and will not allow me to use the phone:  Temperature iPhone needs down before you can use.It is cold and it really can’t cool down any further. I sent a message for a phone call from the seller and it said connected but I never received a call. I also sent a message via chat but that is very cumbersome and instead of just letting me simply state the problem, I had to go through a set of questions that were not relevant.',
 'the os is flickering . charging is not staying long enough. lots of issues',
 'After all data transfer from old iPhone was completed you are asked to contact carrier to activate your phone. Carrier can not activate based on IMEI because it’s not registered on their system. This is the second one I purchased with the same problem thinking first was a one off but problem seem to be with all phones. Went to Apple Store and even they where una

In [259]:
page_number = 1

url = 'https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber='

In [266]:
num_reviews = 25

def get_reviews(num_reviews):
    page_number = 1
    reviews = []

    while len(reviews) <= num_reviews:
        page_url = url + str(page_number)
        print(page_url)

        # no reviews means we've exhausted all reviews
        page_reviews = get_rated_reviews(page_url, num_reviews)

        if len(page_reviews) == 0:
            break

        reviews += page_reviews

        page_number += 1

    # shave off extra reviews coming from the last page
    reviews = reviews[:num_reviews]

    # convert list of strings to list of dictionaries and add rating
    # TODO: 

    return reviews

https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=1
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=2
https://www.amazon.ca/product-reviews/B0BN72FYFG/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=3


In [267]:
reviews

['To soft',
 'I had bought a refurbished iPhone 14 that was unlocked and was having troubles setting up the eSIM (because iphone14 and up are eSIM only) through various trials and errors we had looked up the IEMI number up on T-Mobile data base and it had been reported as stolen through xfinity, now I have a phone that I can’t uses because of this, unbelievable.',
 "This was the worst decision. I got the phone and the screen did not work. The screen is not apple and can tell it doesn't correctly fit. Even if I replaced it, T-Mobile said I would have the same problem. Don't buy from 3rd party. This was a scam.",
 'The screen often goes black and the phone freezes. This phone must have been severely dropped or soaked in water. It is unethical to sell such a phone.',
 'I love my new phone except for one major issue, the front camera will not focus and every picture is blurry! Super upsetting especially since this phone is supposed to take amazing pictures. I’d love to replace it but don’t

In [268]:
len(reviews)

25